In [1]:
# please put all the package used here
from collections import Counter
import pandas as pd
import csv
import pprint
import numpy as np
import warnings
import requests
import json
warnings.filterwarnings('ignore')
import psycopg2
import psycopg2.extras
import time
import ast # convert string to dictionary
from scipy import stats
from scipy.stats import zscore
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns  
sns.set_context('notebook')    
import statsmodels.formula.api as smf
import math as ma
from scipy import stats
import statsmodels.api as sm
import shapefile as shp
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon



In [7]:
data = pd.read_csv('exact_matching.csv')

data_CarSharingPods = pd.read_csv('new_CarSharingPods.csv')

In [8]:
data.head()

,area_id,area_name,land_area,population,number_of_dwellings,population_density,dwelling_density,transport_density,ZP,ZD,...,retail_trade_z,accommodation_and_food_services_z,health_care_and_social_assistance_z,education_and_training_z,arts_and_recreation_services_z,carpods_num,busstops_num,librarybookstore_num,parksplaygrounds_num,trainstation_num
0,102011028,Avoca Beach - Copacabana,643.8,7590,2325,11.789376,3.611370,0.0,-1.127443,-0.953681,...,-0.598852,0.085877,0.847794,-0.394417,1.460536,0.0,60.0,0.0,0.0,0.0
1,102011029,Box Head - MacMasters Beach,3208.6,10986,3847,3.423923,1.198965,0.0,-0.695131,-0.271481,...,-0.664034,-0.292180,-0.370526,-0.404917,1.975785,0.0,111.0,0.0,0.0,0.0
2,102011030,Calga - Kulnura,76795.1,4841,1575,0.063038,0.020509,0.0,-1.477391,-1.289851,...,-1.141346,-0.427656,-1.616320,0.252039,0.801932,0.0,88.0,0.0,0.0,0.0
3,102011031,Erina - Green Point,3379.3,14237,4450,4.213003,1.316841,0.0,-0.281277,-0.001201,...,1.638521,0.017513,1.328059,-0.236636,-0.452820,0.0,145.0,0.0,0.0,0.0
4,102011032,Gosford - Springfield,1691.2,19385,6373,11.462275,3.768330,0.0,0.374065,0.860737,...,0.995075,0.085341,1.652430,0.345641,-0.062011,0.0,180.0,0.0,0.0,0.0


In [9]:
sf = shp.Reader("Sydney2_2016_AUST.shp")
data_CarSharingPods['area_id']=0  # foreign key of another table

count = 0
for shape in sf.shapeRecords():
    polygon = Polygon([(i[0],i[1]) for i in shape.shape.points[:]])
    # calculate number of carpods in each area
    for i in range(len(data_CarSharingPods)):
        lat = data_CarSharingPods['latitude'][i]
        lng = data_CarSharingPods['longitude'][i]
        point = Point(lng, lat)
        if polygon.contains(point) and data_CarSharingPods['area_id'][i] == 0:
            data_CarSharingPods['area_id'][i] = data['area_id'][count]
        elif polygon.contains(point) and data_CarSharingPods['area_id'][i] != 0:
            print("wrong!!!")
            print(data_CarSharingPods['area_id'][i])
            print(data['area_id'][count])
    count += 1    

In [15]:
data_CarSharingPods.to_csv("new_CarSharingPods.csv",index=False)

In [16]:
data_bus = pd.read_csv('BusStops.csv')
sf = shp.Reader("Sydney2_2016_AUST.shp")
data_bus['area_id']=0  # foreign key of another table

count = 0
for shape in sf.shapeRecords():
    polygon = Polygon([(i[0],i[1]) for i in shape.shape.points[:]])
    # calculate number of carpods in each area
    for i in range(len(data_bus)):
        lat = data_bus['stop_lat'][i]
        lng = data_bus['stop_lon'][i]
        point = Point(lng, lat)
        if polygon.contains(point) and data_bus['area_id'][i] == 0:
            data_bus['area_id'][i] = data['area_id'][count]
        elif polygon.contains(point) and data_bus['area_id'][i] != 0:
            print(" wrong!!!")
            print(data_bus['area_id'][i])
            print(data['area_id'][count])
    count += 1    

In [17]:
data_bus.to_csv("BusStops.csv",index=False) 

In [18]:
data_lib = pd.read_csv('LibraryBookstore.csv')
sf = shp.Reader("Sydney2_2016_AUST.shp")
data_lib['area_id']=0  # foreign key of another table

count = 0
for shape in sf.shapeRecords():
    polygon = Polygon([(i[0],i[1]) for i in shape.shape.points[:]])
    # calculate number of carpods in each area
    for i in range(len(data_lib)):
        lat = data_lib['GPS_lat'][i]
        lng = data_lib['GPS_lng'][i]
        point = Point(lng, lat)
        if polygon.contains(point) and data_lib['area_id'][i] == 0:
            data_lib['area_id'][i] = data['area_id'][count]
        elif polygon.contains(point) and data_lib['area_id'][i] != 0:
            print("wrong!!!")
            print(data_lib['area_id'][i])
            print(data['area_id'][count])
    count += 1    

In [19]:
data_lib.to_csv("LibraryBookstore.csv",index=False) 

In [20]:
data_train = pd.read_csv('TrainStation.csv')
sf = shp.Reader("Sydney2_2016_AUST.shp")
data_train['area_id']=0  # foreign key of another table

count = 0
for shape in sf.shapeRecords():
    polygon = Polygon([(i[0],i[1]) for i in shape.shape.points[:]])
    # calculate number of carpods in each area
    for i in range(len(data_train)):
        lat = data_train['GPS_lat'][i]
        lng = data_train['GPS_lng'][i]
        point = Point(lng, lat)
        if polygon.contains(point) and data_train['area_id'][i] == 0:
            data_train['area_id'][i] = data['area_id'][count]
        elif polygon.contains(point) and data_train['area_id'][i] != 0:
            print("wrong!!!")
            print(data_train['area_id'][i])
            print(data['area_id'][count])
    count += 1    
data_train.to_csv("TrainStation.csv",index=False) 

In [25]:
data_park = pd.read_csv('ParksPlaygrounds.csv')
data_park['area_id']=0  # foreign key of another table
sf = shp.Reader("Sydney2_2016_AUST.shp")
count = 0
for shape in sf.shapeRecords():
    polygon = Polygon([(i[0],i[1]) for i in shape.shape.points[:]])
    # calculate number of carpods in each area
    for i in range(len(data_park)):
        lat = data_park['GPS_lat'][i]
        lng = data_park['GPS_lng'][i]
        point = Point(lng, lat)
        if polygon.contains(point) and data_park['area_id'][i] == 0:
            data_park['area_id'][i] = data['area_id'][count]
        elif polygon.contains(point) and data_bus['area_id'][i] != 0:
            print("wrong!!!")
            print(data_bus['area_id'][i])
            print(data['area_id'][count])
    count += 1   
data_park.to_csv("ParksPlaygrounds.csv",index=False)